# Introduction

In this notebook, we present how we scraped data from the websites, specifically for list of Hall of Fame inductees, and seasonal awards. The section Web-Scrape is divided into two parts. The first part, we use package lxml.html, while we use the package BeautifulSoup in the second.

# Web-Scrape

## First Part: Using lxml.html

In this subsection, we show how we scrape the data from the websites using package lxml.html. This method is modified from the webpage https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

### Import

In [1]:
import numpy as np
import requests
import lxml.html as lh
import pandas as pd
import re

### Hall of Fame Inductees

We first scrape the content of the wekipedia page and then extract the elements of the table for Hall of Fame inductees

In [2]:
# Scrape the list of Hall of Fame inductees from wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_players_in_the_Naismith_Memorial_Basketball_Hall_of_Fame'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

We then use the length of the elements in each row to identify the part we want.

In [3]:
# total number of rows in the table is 201
len(tr_elements)

201

In [4]:
# there should be 5 elements in each row
list_outside = []
for i in range(0,201):
    if len(tr_elements[i]) != 5:
        list_outside = list_outside + [i]

print(str(list_outside)+' are not in the table!')
    

[192, 193, 194, 195, 196, 197, 198, 199, 200] are not in the table!


In [5]:
# this is actually the last row in the table we would like to extract
tr_elements[191].text_content()

'\n2018\nCharlie Scott\nG\nNBA champion (1976)3× NBA All-Star (1973–1975)2× ABA All-Star (1971, 1972)All-ABA First Team (1971)All-ABA Second Team (1972)ABA Rookie of the Year (1971)ABA All-Rookie First Team (1971)ABA All-Time Team2× Consensus second-team All-American (1969, 1970)ACC Athlete of the Year (1970)College Basketball Hall of Fame (2015)Summer Olympics \xa0Gold: 1968\n[174]\n'

In [6]:
tr_elements = tr_elements[0:192]

We now create a dictionary by using the header and table content, in order to create a dataframe in the end

In [7]:
# The first row should be the header
[t.text_content() for t in tr_elements[0]]

['Year\n', 'Inductees\n', 'Pos.\n', 'Achievements\n', 'Ref.\n']

In [8]:
# Extract the header and create a list
col = []
i=0
for t in tr_elements[0]:
    i = i+1
    name = t.text_content().replace('\n','')
    print('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Year"
2:"Inductees"
3:"Pos."
4:"Achievements"
5:"Ref."


In [9]:
# check list 'col,' the first item in each tuple is the column name and the rest is the data
col

[('Year', []),
 ('Inductees', []),
 ('Pos.', []),
 ('Achievements', []),
 ('Ref.', [])]

In [10]:
# collect the content of the table in the dictionary
for j in range(1,len(tr_elements)):
    T = tr_elements[j]
    i=0
    for t in T.iterchildren():
        data = t.text_content()
        col[i][1].append(data)
        i = i+1
        

In [11]:
type(col)

list

In [12]:
# create the dictionary out of the list of tuple, 'col'
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [13]:
df.head()

,Year,Inductees,Pos.,Achievements,Ref.
0,1959,Chuck Hyatt,G,"National championship (Pittsburgh, 1928, 1930)...",[3]\n
1,1959,Hank Luisetti,F,3 Pacific Coast Conference championships (Stan...,[4]\n
2,1959,George Mikan,C,"All-America (DePaul, 1944–45); All-NBA First-T...",[5]\n
3,1959,John Schommer,G,"Big Ten Championships (Chicago, 1907–09); All-...",[6]\n
4,1960,Vic Hanson,G,"Helms Foundation Championship (Syracuse, 1926)...",[7]\n


In [14]:
export_csv = df.to_csv (r'../data/Hall-of-Fame.csv', index = None, header=True)

### NBA Final MVP

We scrape the content of the wekipedia page and then extract the elements of the table for NBA Final MVP

In [15]:
# Scrape the data from wikipedia
url = 'https://en.wikipedia.org/wiki/Bill_Russell_NBA_Finals_Most_Valuable_Player_Award'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

We then use the length of the elements in each row to identify the part we want.

In [16]:
# total number of rows in the table is 114
len(tr_elements)

114

In [17]:
# there should be 5 elements in each row
list_outside = []
for i in range(0,114):
    if len(tr_elements[i]) != 5:
        list_outside = list_outside + [i]

print(str(list_outside)+' are not in the table!')
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113] are not in the table!


In [18]:
# The header of the table is in row 16
tr_elements[16].text_content()

'\nYear\n\nPlayer\n\nPosition\n\nNationality\n\nTeam\n'

In [19]:
# The last row of the table in in row 66
tr_elements[66].text_content()

'\n2018\n\nKevin Durant^ (2)\n\nForward\n\n\xa0United States\n\nGolden State Warriors (4)\n'

In [20]:
# keep only from 16 to 66
tr_elements = tr_elements[16:(67)]

In [21]:
# The first row should be the header
[t.text_content() for t in tr_elements[0]]

['Year\n', 'Player\n', 'Position\n', 'Nationality\n', 'Team\n']

In [22]:
# Extract the header and create a list
col = []
i=0
for t in tr_elements[0]:
    i = i+1
    name = t.text_content().replace('\n','')
    print('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Year"
2:"Player"
3:"Position"
4:"Nationality"
5:"Team"


In [23]:
# Store all the data in the list
for j in range(1,len(tr_elements)):
    T = tr_elements[j]
    i=0
    for t in T.iterchildren():
        data = t.text_content()
        col[i][1].append(data)
        i = i+1

In [24]:
# create the dictionary out of the list of tuple, 'col'
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [25]:
df.head()

,Year,Player,Position,Nationality,Team
0,1969\n,Jerry West*\n,Guard\n,United States\n,Los Angeles Lakers §\n
1,1970\n,Willis Reed*\n,Center/Forward\n,United States\n,New York Knicks\n
2,1971\n,Lew Alcindor*[a]\n,Center\n,United States\n,Milwaukee Bucks\n
3,1972\n,Wilt Chamberlain*\n,Center\n,United States\n,Los Angeles Lakers (2)\n
4,1973\n,Willis Reed* (2)\n,Center/Forward\n,United States\n,New York Knicks (2)\n


We now start to clean the data in the dataframe df

In [26]:
# for year, take only the integer part
regex = re.compile(r'^(\d{4})$')
df['Year'] = df['Year'].apply(lambda x: int(regex.findall(x)[0]))

In [27]:
# for the Player, take only the name part
df['Player'] = df['Player'].str.replace('*','')
df['Player'] = df['Player'].str.replace('^','')
df['Player'] = df['Player'].str.replace('\n','')

In [28]:
# get rid of any footnote in the Player column
df['Player'] = df['Player'].apply(lambda x: re.findall(r'[a-zA-Z\s\'\-]+',x)[0])

In [29]:
# get rid of the space at the end
for i in range(0,df.shape[0]):
    if df.loc[i,'Player'][-1] == ' ':
        df.loc[i,'Player'] = df.loc[i,'Player'][:-1]
        

In [30]:
# get rid of other column
df.drop(['Position','Nationality','Team'],axis=1,inplace=True)

In [31]:
Final_MVP = df.groupby('Player').count().reset_index()

In [32]:
Final_MVP.rename(index=str,columns={'Year':'Final_MVP'},inplace=True)

In [33]:
# The dataframe include the Player's name and the number of times he is awarded final MVP
Final_MVP.head()

,Player,Final_MVP
0,Andre Iguodala,1
1,Bill Walton,1
2,Cedric Maxwell,1
3,Chauncey Billups,1
4,Dennis Johnson,1


In [34]:
export_csv = Final_MVP.to_csv (r'../data/Final-MVP.csv', index = None, header=True)

### NBA All-Star MVP

We scrape the content of the wekipedia page and then extract the elements of the table for NBA All-Star MVP

In [35]:
url = 'https://en.wikipedia.org/wiki/NBA_All-Star_Game_Most_Valuable_Player_Award'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

We then use the length of the elements in each row to identify the part we want.

In [36]:
# total number of rows in the table is 137
len(tr_elements)

139

In [37]:
# there should be 5 elements in each row but sometimes 4 (e.g. 2009 and 2000)
list_outside = []
for i in range(0,137):
    if len(tr_elements[i]) != 5:
        list_outside = list_outside + [i]

print(str(list_outside)+' are not in the table!')
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 25, 60, 66, 68, 78, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136] are not in the table!


In [38]:
# The header of the table is in row 15
tr_elements[15].text_content()

'\nSeason\n\nPlayer\n\nPosition\n\nNationality\n\nTeam\n'

In [39]:
# The last row of the table in in row 65
tr_elements[87].text_content()

'\n2018\n\nLeBron James^ (3)\n\nForward\n\n\xa0United States\n\nCleveland Cavaliers (4)\n'

In [40]:
# The rows in the table with 4 elements
for i in [25,60,66,68,78]:
    print([t.text_content() for t in tr_elements[i]])

['Bob Pettit* (3)\n', 'Forward/Center\n', '\xa0United States\n', 'St. Louis Hawks (3)\n']
['Karl Malone* (2)\n', 'Forward\n', '\xa0United States\n', 'Utah Jazz (3)\n']
['1999\n', "Not awarded as the game was canceled due to the league's lockout.[3]\n"]
['Tim Duncan\n', 'Forward/Center\n', '\xa0United States[c]\n', 'San Antonio Spurs (2)\n']
["Shaquille O'Neal* (3) [e]\n", 'Center\n', '\xa0United States\n', 'Phoenix Suns\n']


In [41]:
# keep only from 15 to 88
tr_elements = tr_elements[15:88]

In [42]:
# Extract the header and create a list
col = []
i=0

for t in tr_elements[0]:
    i = i+1
    name = t.text_content().replace('\n','')
    print('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Season"
2:"Player"
3:"Position"
4:"Nationality"
5:"Team"


In [43]:
# Store all the data in the list
for j in range(1,len(tr_elements)):
    T = tr_elements[j]   
    if len(T) == 5: # if row is of size 5, this belongs to our table
        i=0
    elif len(T) == 4: # if row is of size 4, that is the year when there are two all-star MVP      
        i=1
        col[0][1].append(year)
    else: # if row is not of size 5 or 4, the //tr data is not from our table (for year 1999 specifically)
        continue
    
    for t in T.iterchildren():
        if i == 0:
            year = t.text_content()
            
        data = t.text_content()
        col[i][1].append(data)
        i = i+1

In [44]:
# create the dictionary out of the list of tuple, 'col'
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [45]:
df.head()

,Season,Player,Position,Nationality,Team
0,1951\n,Ed Macauley*\n,Center/Forward\n,United States\n,Boston Celtics\n
1,1952\n,Paul Arizin*\n,Forward/Guard\n,United States\n,Philadelphia Warriors\n
2,1953\n,George Mikan*\n,Center\n,United States\n,Minneapolis Lakers\n
3,1954\n,Bob Cousy*\n,Guard\n,United States\n,Boston Celtics (2)\n
4,1955\n,Bill Sharman*\n,Guard\n,United States\n,Boston Celtics (3)\n


In [46]:
# for year, take only the integer part
regex = re.compile(r'^(\d{4})\S*$')
df['Season'] = df['Season'].apply(lambda x: int(regex.findall(x)[0]))

In [47]:
# for the Player, take only the name part
df['Player'] = df['Player'].str.replace('*','')
df['Player'] = df['Player'].str.replace('^','')
df['Player'] = df['Player'].str.replace('\n','')

In [48]:
# get rid of any footnote
df['Player'] = df['Player'].apply(lambda x: re.findall(r'[a-zA-Z\s\'\-]+',x)[0])

In [49]:
# get rid of the space at the end
for i in range(0,df.shape[0]):
    if df.loc[i,'Player'][-1] == ' ':
        df.loc[i,'Player'] = df.loc[i,'Player'][:-1]
        

In [50]:
# get rid of other column
df.drop(['Position','Nationality','Team'],axis=1,inplace=True)

In [51]:
AllStar_MVP = df.groupby('Player').count().reset_index()

In [52]:
AllStar_MVP.rename(index=str,columns={'Season': 'AllStar_MVP'},inplace=True)

In [53]:
# The dataframe include the Player's name and the number of times he is awarded all-star MVP
AllStar_MVP.head()

,Player,AllStar_MVP
0,Adrian Smith,1
1,Allen Iverson,2
2,Anthony Davis,1
3,Bill Russell,1
4,Bill Sharman,1


In [54]:
export_csv = AllStar_MVP.to_csv (r'../data/All-Star-MVP.csv', index = None, header=True)

### Defensive Player Of The Year

We scrape the content of the wekipedia page and then extract the elements of the table for defensive player of the year.

In [55]:
url = 'https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

We then use the length of the elements in each row to identify the part we want.

In [56]:
# total number of rows in the table is 63
len(tr_elements)

63

In [57]:
# there should be 5 elements in each row but sometimes 4 (e.g. 2009 and 2000)
list_outside = []
for i in range(0,63):
    if len(tr_elements[i]) != 5:
        list_outside = list_outside + [i]

print(str(list_outside)+' are not in the table!')
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 53, 56, 57, 58, 59, 60, 61, 62] are not in the table!


In [58]:
# The header of the table is in row 14
tr_elements[14].text_content()

'\nSeason\n\nPlayer\n\nPosition\n\nNationality\n\nTeam\n'

In [59]:
# The last row of the table in in row 50
tr_elements[50].text_content()

'\n2017–18\n\nRudy Gobert^\n\nCenter\n\n\xa0France\n\nUtah Jazz\n'

In [60]:
# keep only from 14 to 50
tr_elements = tr_elements[14:51]

In [61]:
# Extract the header and create a list
col = []
i=0

for t in tr_elements[0]:
    i = i+1
    name = t.text_content().replace('\n','')
    print('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Season"
2:"Player"
3:"Position"
4:"Nationality"
5:"Team"


In [62]:
# Store all the data in the list
for j in range(1,len(tr_elements)):
    T = tr_elements[j]
    i=0
    for t in T.iterchildren():
        data = t.text_content()
        col[i][1].append(data)
        i = i+1

In [63]:
# create the dictionary out of the list of tuple, 'col'
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [64]:
df.head()

,Season,Player,Position,Nationality,Team
0,1982–83\n,Sidney Moncrief\n,Guard\n,United States\n,Milwaukee Bucks\n
1,1983–84\n,Sidney Moncrief (2)\n,Guard\n,United States\n,Milwaukee Bucks\n
2,1984–85\n,Mark Eaton\n,Center\n,United States\n,Utah Jazz\n
3,1985–86\n,Alvin Robertson\n,Guard\n,United States\n,San Antonio Spurs\n
4,1986–87\n,Michael Cooper[a]\n,Guard/Forward\n,United States\n,Los Angeles Lakers\n


In [65]:
# for year, take only the integer part
regex = re.compile(r'^(\d{4})')
df['Season'] = df['Season'].apply(lambda x: int(regex.findall(x)[0])+1)

In [66]:
df.tail()

,Season,Player,Position,Nationality,Team
31,2014,Joakim Noah^\n,Center\n,France[f]\n,Chicago Bulls\n
32,2015,Kawhi Leonard^\n,Forward\n,United States\n,San Antonio Spurs\n
33,2016,Kawhi Leonard^ (2)\n,Forward\n,United States\n,San Antonio Spurs\n
34,2017,Draymond Green^[a]\n,Forward\n,United States\n,Golden State Warriors\n
35,2018,Rudy Gobert^\n,Center\n,France\n,Utah Jazz\n


In [67]:
# for the Player, take only the name part
df['Player'] = df['Player'].str.replace('*','')
df['Player'] = df['Player'].str.replace('^','')
df['Player'] = df['Player'].str.replace('\n','')

In [68]:
# get rid of any footnote
df['Player'] = df['Player'].apply(lambda x: re.findall(r'[a-zA-Z\s\'\-]+',x)[0])

In [69]:
# get rid of the space at the end
for i in range(0,df.shape[0]):
    if df.loc[i,'Player'][-1] == ' ':
        df.loc[i,'Player'] = df.loc[i,'Player'][:-1]
        

In [70]:
# get rid of other column
df.drop(['Position','Nationality','Team'],axis=1,inplace=True)

In [71]:
df.rename(index = str, columns = {'Season':'Year'}, inplace = True)

In [72]:
# The dataframe include the Player's name and the number of times he is awarded defensive player of the year
DPOY = df.groupby('Player').count().reset_index().rename(index = str, columns = {'Year':'DPOY'})

In [73]:
DPOY.head()

,Player,DPOY
0,Alonzo Mourning,2
1,Alvin Robertson,1
2,Ben Wallace,4
3,David Robinson,1
4,Dennis Rodman,2


In [74]:
export_csv = DPOY.to_csv (r'../data/DPOY.csv', index = None, header=True)

### NBA Teams That Attend Finals And Win Championships

Here we present how we construct the data for players whose teams attend finals or even win the championships. It is a three-step process: 
    (1) We scrape the table for the teams that attend finals from wikipedia, and construct dataframe 'NBA_final' for teams which attend finals and win championships in each year.
    (2) We scrape the table for full names for the NBA teams and corresponding abbreviation, and replace the teams' name in 'NBA_final' with abbreviation.
    (3) We use the player's seasonal statistics and 'NBA_final' to construct the desired dataframe that include players name and number of times the player's team attend finals and wind the championships.

#### Step 1: Constuct DataFrame For Teams Which Attend Finals And Win Championships

In [75]:
url = 'https://en.wikipedia.org/wiki/List_of_NBA_champions'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

We then use the length of the elements in each row to identify the part we want.

In [76]:
# total number of rows in the table is 154
len(tr_elements)

154

In [77]:
# there should be 7 elements in each row
list_outside = []
for i in range(0,154):
    if len(tr_elements[i]) != 7:
        list_outside = list_outside + [i]

print(str(list_outside)+' are not in the table!')
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153] are not in the table!


In [78]:
# The header of the table is in row 12
tr_elements[12].text_content()

'\nYear\n\nWestern Champion\n\nCoach\n\nResult\n\nEastern Champion\n\nCoach\n\nReference\n'

In [79]:
# The last row of the table in in row 84
tr_elements[84].text_content()

'\n2018\nGolden State Warriors (2) (10, 6–4)\nSteve Kerr\n4–0\nCleveland Cavaliers (4) (5, 1–4)\nTyronn Lue\n[91]\n'

In [80]:
# keep only from 12 to 84
tr_elements = tr_elements[12:(84+1)]

In [81]:
# Extract the header and create a list. Please note that since the column name coach appears twice, 
# we distinguish them by appending the column's name with  the column's number index
col = []
i=0
for t in tr_elements[0]:
    i = i+1
    name = t.text_content().replace('\n','')
    print('%d:"%s"' %(i,name))
    col.append((name+str(i),[]))

1:"Year"
2:"Western Champion"
3:"Coach"
4:"Result"
5:"Eastern Champion"
6:"Coach"
7:"Reference"


In [82]:
# Store all the data in the list
for j in range(1,len(tr_elements)):
    T = tr_elements[j]
    if len(T) != 7:
        break
    i=0
    for t in T.iterchildren():
        data = t.text_content()
        col[i][1].append(data)
        i = i+1

In [83]:
# create the dictionary out of the list of tuple, 'col'
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [84]:
df.head()

,Year1,Western Champion2,Coach3,Result4,Eastern Champion5,Coach6,Reference7
0,1947,"Chicago Stags (1) (1, 0–1)",Harold Olsen,1–4,"Philadelphia Warriors (2) (1, 1–0)",Eddie Gottlieb,[14]\n
1,1948,"Baltimore Bullets (2) (1, 1–0)",Buddy Jeannette,4–2,"Philadelphia Warriors (1) (2, 1–1)",Eddie Gottlieb,[15]\n
2,1949,"Minneapolis Lakers (2) (1, 1–0)",John Kundla,4–2,"Washington Capitols (1) (1, 0–1)",Red Auerbach,[16]\n
3,1950,"Minneapolis Lakers (1) [a] (2, 2–0)",John Kundla,4–2,"Syracuse Nationals (1) (1, 0–1)",Al Cervi,[17][18]\n
4,1951,"Rochester Royals (2) (1, 1–0)",Les Harrison,4–3,"New York Knicks (3) (1, 0–1)",Joe Lapchick,[19]\n


In [85]:
# drop information that we don't need and replace the column names
df.drop(['Coach3','Coach6','Reference7'],axis=1,inplace=True)
df.rename(columns={'Year1':'Year', 'Western Champion2':'W', 'Result4':'Results','Eastern Champion5': 'E'},inplace=True)

In [86]:
# Clean Year
regex = re.compile(r'^(\d{4})\S*$')
df['Year'] = df['Year'].apply(lambda x: int(regex.findall(x)[0]))

In [87]:
# Clean W and E
regex = re.compile(r'^([a-zA-Z.\s]+)')
df['W'] = df['W'].apply(lambda x: regex.findall(x)[0])
df['E'] = df['E'].apply(lambda x: regex.findall(x)[0])

# drop the last white space
df['W'] = df['W'].apply(lambda x: x[:-1] if x[-1]==' ' else x[:])
df['E'] = df['E'].apply(lambda x: x[:-1] if x[-1]==' ' else x[:])

In [88]:
# create new columns, 'W Win' and 'E Win' according to 'Results'
regex = re.compile(r'^(\d)\S(\d)*')
df['W Win'] = df['Results'].apply(lambda x: int(regex.findall(x)[0][0]))
df['E Win'] = df['Results'].apply(lambda x: int(regex.findall(x)[0][1]))

In [89]:
# Create new columns 'Champion,' based on 'W Win' and 'E Win'
df['Champion'] = df.apply(lambda x: x['W'] if x['W Win'] > x['E Win'] else x['E'],axis=1)

In [90]:
# drop columns 'Results,' 'W Win,' and 'E Win'
df.drop(['Results','W Win','E Win'],axis=1,inplace = True)
df.head()

,Year,W,E,Champion
0,1947,Chicago Stags,Philadelphia Warriors,Philadelphia Warriors
1,1948,Baltimore Bullets,Philadelphia Warriors,Baltimore Bullets
2,1949,Minneapolis Lakers,Washington Capitols,Minneapolis Lakers
3,1950,Minneapolis Lakers,Syracuse Nationals,Minneapolis Lakers
4,1951,Rochester Royals,New York Knicks,Rochester Royals


In [91]:
NBA_final = df.reset_index().copy()

#### Step 2: Scrape The Table For Name-Abbreviation Conversion

In [92]:
url = 'http://www.shrpsports.com/nba/explain.htm'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [93]:
# total number of rows in the table is 178
len(tr_elements)

178

In [94]:
# there should be 3 elements in each row
list_outside = []
for i in range(0,178):
    if len(tr_elements[i]) != 3:
        list_outside = list_outside + [i]

print(str(list_outside)+' are not in the table!')
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104] are not in the table!


In [95]:
# The header of the table is in row 3
tr_elements[105].text_content()

'\nAtlanta      AtlAtlanta Hawks (1968-69 - present)'

In [96]:
# The last row of the table in in row 84
tr_elements[177].text_content()

'\nWaterloo     WatWaterloo Hawks (1949-50)'

In [97]:
# keep only from 105 to 177
tr_elements = tr_elements[105:(177+1)]

In [98]:
# The website doesn't have header, define by myself
col = []
col.append(('Long Abb',[]))
col.append(('Abb',[]))
col.append(('Name',[]))

In [99]:
# Store all the data in the list
for j in range(0,len(tr_elements)):
    T = tr_elements[j]
    i=0
    for t in T.iterchildren():
        data = t.text_content()
        col[i][1].append(data)
        i = i+1

In [100]:
# create the dictionary out of the list of tuple, 'col'
Dict={title:column for (title,column) in col}
Team_Abb=pd.DataFrame(Dict)

In [101]:
Team_Abb.head()

,Long Abb,Abb,Name
0,Atlanta,Atl,Atlanta Hawks (1968-69 - present)
1,Anderson,And,Anderson Packers (1949-50)
2,Bal Bullets,BlB,Baltimore Bullets (1st team) (1947-48 - 1953-54)
3,Baltimore,Bal,Baltimore Bullets (2nd team) (1963-64 - 1972-73)
4,Boston,Bos,Boston Celtics (1946-47 - present)


In [102]:
# Drop 'Long Abb'
Team_Abb.drop(['Long Abb'],axis=1,inplace=True)

In [103]:
# Make all the Abbreviation uppercase
Team_Abb['Abb'] = Team_Abb['Abb'].apply(lambda x: x.upper())
Team_Abb['Abb'] = Team_Abb['Abb'].apply(lambda x: x[:-1] if x[-1]==' ' else x[:])
Team_Abb.head()

,Abb,Name
0,ATL,Atlanta Hawks (1968-69 - present)
1,AND,Anderson Packers (1949-50)
2,BLB,Baltimore Bullets (1st team) (1947-48 - 1953-54)
3,BAL,Baltimore Bullets (2nd team) (1963-64 - 1972-73)
4,BOS,Boston Celtics (1946-47 - present)


In [104]:
# take only the team's name
regex = re.compile(r'^([a-zA-Z.\s\-]+)')
Team_Abb['Name'] = Team_Abb['Name'].apply(lambda x: regex.findall(x)[0])
# drop the last white space
Team_Abb['Name'] = Team_Abb['Name'].apply(lambda x: x[:-1] if x[-1]==' ' else x[:])

To exploit dataframe 'Team_Abb' for converting the teams' names in dataframe 'NBA_final', we first compare if there is any team in 'NBA_final' not in 'Team_Abb' and deal with the entry.

In [105]:
NBA_final_team_set = set(NBA_final['W'].tolist()+NBA_final['E'].tolist())
Team_Abb_Name_set = set(Team_Abb['Name'].tolist())
print(NBA_final_team_set.difference(Team_Abb_Name_set))

{'Fort Wayne Pistons', 'St. Louis Hawks', 'New York Knicks', 'Portland Trail Blazers', 'Oklahoma City Thunder'}


In [106]:
# We find the corresponding team and correct the name in dataframe 'Team_Abb'
Team_Abb.at[Team_Abb.index[Team_Abb['Name'] == 'New York Knick'],'Name'] = 'New York Knicks'
Team_Abb.at[Team_Abb.index[Team_Abb['Name'] == 'Portland TrailBlazers'],'Name'] = 'Portland Trail Blazers'
Team_Abb.at[Team_Abb.index[Team_Abb['Name'] == 'Ft Wayne Pistons'],'Name'] = 'Fort Wayne Pistons'
Team_Abb.at[Team_Abb.index[Team_Abb['Name'] == 'Oklahoma City'],'Name'] = 'Oklahoma City Thunder'
Team_Abb.at[Team_Abb.index[Team_Abb['Name'] == 'St Louis Hawks'],'Name'] = 'St. Louis Hawks'
Team_Abb_Name_set = set(Team_Abb['Name'].tolist())
print(NBA_final_team_set.difference(Team_Abb_Name_set))

set()


To make the conversion from teams' full names to the corresponding abbreviations, we seperate the names that appear only once in NBA history and the ones that apprear more than onece, and establish a dictionary for the first one but deal with the latter by hand.

In [107]:
# Teams that appear more than once
temp = (Team_Abb.groupby('Name').count()>1)
team_recurring_list =  temp.index[temp.Abb].tolist()
print('Recurring team:' + str(team_recurring_list))

# Teams that only appear once
team_once_list =list(Team_Abb_Name_set)
for x in team_recurring_list:
    team_once_list.remove(x) 

Recurring team:['Baltimore Bullets', 'Denver Nuggets']


In [108]:
# Baltimore Bullets appear twice
Team_Abb_recuring_index = pd.Index([])
for x in team_recurring_list:
    Team_Abb_recuring_index = Team_Abb_recuring_index.append(Team_Abb.index[(Team_Abb['Name'] == x)])
    
Team_Abb_once_index = Team_Abb.index.drop(Team_Abb_recuring_index)
Team_Abb_recuring = Team_Abb.iloc[Team_Abb_recuring_index].copy()
Team_Abb_once = Team_Abb.iloc[Team_Abb_once_index].copy()

In [109]:
# For the team whose name only appear once
dict_temp = Team_Abb_once.set_index('Name').T.to_dict('list')
NBA_final.replace(dict_temp,inplace=True)

In [110]:
# For the 'Baltimore Bullets'
# BLB before 1960, BAL after 1960
NBA_final[(NBA_final['W'] == 'Baltimore Bullets') |(NBA_final['E'] == 'Baltimore Bullets') | (NBA_final['Champion'] == 'Baltimore Bullets')]

,index,Year,W,E,Champion
1,1,1948,Baltimore Bullets,PHW,Baltimore Bullets
24,24,1971,MIL,Baltimore Bullets,MIL


In [111]:
# replace the frist row with BLB and the 24th row with BAL
NBA_final.at[1,'W'] = 'BLB'
NBA_final.at[1,'Champion'] = 'BLB'
NBA_final.at[24,'E'] = 'BAL'

In [112]:
# Fpr the 'Denver Nuggets'
# never attended NBA finals
NBA_final[(NBA_final['W'] == 'Denver Nuggets') |(NBA_final['E'] == 'Denver Nuggets') | (NBA_final['Champion'] == 'Denver Nuggets')]

,index,Year,W,E,Champion


Thus far, we have made the data frame NBA_final with column 'Year', 'W' (stands for Western Champion), 'E' (stands for Eastern Champion), and 'Champion' (stands for 'Championship'). All the teams are labeled using the abbreviations we scrape online.

#### Constuct The Data Frame For Players

We now use the seaonal statistics data, Seasons_stats.csv, to construct the data frame recording the number of times that the players' team attend the final or win the championship.

In [113]:
# import seasonal data
raw_season_stats = pd.read_csv('../data/Seasons_Stats.csv')
raw_season_stats.drop(raw_season_stats.columns[0],axis=1,inplace=True)
# delete *
raw_season_stats['Player'] = raw_season_stats['Player'].str.replace('*','')
# delete blank columns
raw_season_stats.drop(['blanl','blank2'],axis=1,inplace = True)
# Create new feature 'birth_year' in All_season_stats, in order to distinguish the players with the same legal names
raw_season_stats['Birth Year'] = raw_season_stats['Year'] - raw_season_stats['Age']

However, we find that there are discrepancies between the abbreviations in the online-list and those in the Seasons_stats.csv, which we are going to fix.

In [114]:
# The list of team abbrev. in raw-season_Tm
raw_season_Tm_set = set(raw_season_stats['Tm'].dropna())
# The list of team abbrev. in NBA final
NBA_final_Tm_set = set(NBA_final['W'].tolist()+NBA_final['E'].tolist())
# The difference
NBA_final_Tm_difference=sorted(list(NBA_final_Tm_set.difference(raw_season_Tm_set)))
for x in NBA_final_Tm_difference:
    print(Team_Abb[Team_Abb['Abb'] == x])

   Abb                   Name
23  GS  Golden State Warriors
   Abb             Name
38  NJ  New Jersey Nets
   Abb             Name
43  NY  New York Knicks
   Abb               Name
55  SA  San Antonio Spurs
   Abb                    Name
57  SF  San Francisco Warriors


In [115]:
# choose from above
# 'GS' should be 'GSW'
# 'NJ'           'NJN'
# 'NY'           'NYK'
# 'SA'           'SAS'
# 'SF'           'SFW'
NBA_final.replace({'GS':'GSW', 'NJ': 'NJN', 'NY': 'NYK', 'SA': 'SAS', 'SF':'SFW'},inplace=True)
# double check there is no missing Abb 
print(sorted(list(set(NBA_final['W'].tolist()+NBA_final['E'].tolist()).difference(raw_season_Tm_set))))

[]


In [116]:
# import the career_stats
player_stats = pd.read_csv('../data/player_stats.csv')

In [117]:
# define the data frame for the features, 'Final' and 'Champion'
df_feature = player_stats[['Player','Birth Year']].copy()
df_feature['Final'] = 0
df_feature['Champion'] = 0

In [118]:
for year in range(1947,2018):
    Tm_W = NBA_final.at[NBA_final['Year'].index[NBA_final['Year']== year][0],'W']
    Tm_E = NBA_final.at[NBA_final['Year'].index[NBA_final['Year']== year][0],'E']
    Tm_Champ = NBA_final.at[NBA_final['Year'].index[NBA_final['Year']== year][0],'Champion']
    
    # for Western Champion
    temp = raw_season_stats[(raw_season_stats['Year'] == year) & (raw_season_stats['Tm'] == Tm_W)][['Player','Birth Year']]
    for (x,y) in zip(temp['Player'],temp['Birth Year']):
        temp_index = (df_feature['Player'] == x) & (df_feature['Birth Year'] == y)
        if temp_index.sum()>0: 
            df_feature.at[df_feature.index[temp_index][0],'Final'] +=1
        
    # for Eastern Champion
    temp = raw_season_stats[(raw_season_stats['Year'] == year) & (raw_season_stats['Tm'] == Tm_E)][['Player','Birth Year']]
    for (x,y) in zip(temp['Player'],temp['Birth Year']):
        temp_index = (df_feature['Player'] == x) & (df_feature['Birth Year'] == y)
        if temp_index.sum()>0: 
            df_feature.at[df_feature.index[temp_index][0],'Final'] +=1
    
    
    # for Champion
    temp = raw_season_stats[(raw_season_stats['Year'] == year) & (raw_season_stats['Tm'] == Tm_Champ)][['Player','Birth Year']]
    for (x,y) in zip(temp['Player'],temp['Birth Year']):
        temp_index = (df_feature['Player'] == x) & (df_feature['Birth Year'] == y)
        if temp_index.sum()>0: 
            df_feature.at[df_feature.index[temp_index][0],'Champion'] +=1
    

In [119]:
df_feature[(df_feature['Player']=='Kareem Abdul-Jabbar')&(player_stats['Birth Year']==1948)]

,Player,Birth Year,Final,Champion
1445,Kareem Abdul-Jabbar,1948.0,10,6


In [120]:
export_csv = df_feature.to_csv (r'../data/Final-and-Champion.csv', index = None, header=True)

# Second Part

## Using Beautifulsoup

In [121]:
# We Will use requests to get the HTML source code from Wikipedia
# and use BeautifulSoup to parse the HTML.
from bs4 import BeautifulSoup

# This produces a dataframe from a single wikitable on url
# of a particular tableClass with cols = numcols.

def import_wikitable(url, numcols, tableClass):
    website_url = requests.get(url)
    soup = BeautifulSoup(website_url.text, 'lxml')
    # By noting the class of the table in the source code,
    # and the fact that it is the only table we pick it out 
    # using find()
    My_table = soup.find('table',{'class':tableClass})
    
    # Creating lists for each of the columns I know to be in my 
    # wikitable.
    a = {}
    k = 0
    while k < numcols:
        # Dynamically create key
        key = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[k]
        # Calculate value
        value = []
        a[key] = value 
        k += 1

    # Using HTML tags for rows <tr> and elements <td> to iterate
    # through each row of data and append data elements to 
    # their appropriate lists:

    for row in My_table.findAll('tr'):
        #print(row)
        cells = row.findAll(['td','th'])
        if len(cells) > 0: # Only extract table body 
            k = 0
            for x in a:
                a[x].append(cells[k].find(text=True))
                k = k+1
    
    # We should be able to read the header of the table and 
    # assign the proper header names, but I'm not going to right now
    # Now create a dataframe to export
    
    df = pd.DataFrame(a['A'] ,columns = ['A'] )
    a.pop('A')
    for x in a:
        df[x] = a[x] 
    
    df.columns = df.iloc[0]
    
    columnNames = [];
    for x in df.columns:
        columnNames.append(x[0:-1])

    df.columns = columnNames
    return df.drop(0)
    

In [122]:
# Here I call the function on several wikipedia tables.
HOFTable = import_wikitable('https://en.wikipedia.org/wiki/List_of_players_in_the_Naismith_Memorial_Basketball_Hall_of_Fame',
                            5,
                            'wikitable sortable')

AllStarTable = import_wikitable('https://en.wikipedia.org/wiki/List_of_NBA_All-Stars',
                                5,
                                'wikitable sortable')

AllTeamsTable = import_wikitable('https://en.wikipedia.org/wiki/All-NBA_Team',
                                 7,
                                 'wikitable sortable')


In [123]:
# The MVP table is a little tricky, so we need to do some manipulations to get it sorted out

MVPTable = import_wikitable('https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award',
                            5,
                            'wikitable plainrowheaders sortable')

# This just counts the multiplicity of all entries in a list and 
# returns a dictionary of those multiplicities keyed by the list values
def multiplicity(list):
    out = {}
    for y in list:
        indices = [i for i, x in enumerate(list) if x == y]
        out[y] = len(indices)
    return(out)

y = pd.DataFrame.from_dict(multiplicity(MVPTable.drop(["Position", "Season", "Nationality", "Team"], axis = 1)["Player"].tolist()), orient = 'index')

y.index.name = 'Player'

y = y.rename(index=str, columns={"Player": "Player", 0: "MVPs"})

MVPTable = y.reset_index()

# Here I fix an issue in the MVP data in which Kareem's first MVP is under the name Lew Alcindor

#len(set(MVPTable['Player'].tolist()))
MVPTable.loc[MVPTable['Player'] == 'Lew Alcindor', 'Player'] = 'Kareem Abdul-Jabbar'


In [124]:
# We loop over the dict of tables to save each one to its own CSV file.

tables = {"HOFTable":HOFTable, "MVPTable":MVPTable, "AllStarTable":AllStarTable, "AllTeamsTable":AllTeamsTable}

for x in tables:
    tables[x].to_csv('../data/' + x + '.csv', index = False, sep = ',')